Based on Example3.ipynb, we allow holes in the domain 

# One hole

In [17]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Generate computation domain \Omega_1
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(0.0,0.5,0.1).Reverse().Face() # with a hole with r=0.1 in the center 
geo = OCCGeometry(domain, dim = 2)
# Generate a triangular mesh of mesh-size 0.01, modify maxh value for resutls with h=0.03 and h=0.05
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
# Define vector field A and FEM space for solving the Neumann problem, fix the local polynomial degree at p=3
a =100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

# Define trial-function, test-function, and the variation form 
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

# the solution field
gfm.vec.data = a.mat.Inverse() * f.vec

F = A - grad(gfu)
F0x, F1x = F.Diff(x)
F0y, F1y = F.Diff(y)
Draw(A, mesh,vector=True)
Draw(F, mesh,vector=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [18]:
# Calculate the L^2 norm for A and F
sqrt(Integrate(A*A, mesh)), sqrt(Integrate(F*F, mesh))

(199.2433937127805, 118.62320366222475)

## Use dirichlet BC

### This section is to calculate the eigenpairs for H(A)

In [19]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG

start1 = time.time()
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(0.0,0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
# X.ndof
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

# Use FEAST algorithm for computing eigenvalues
seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 19, center = 139; When h=0.03, radius = 29, center = 171; When h=0.05, radius = 41, center = 371
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=41,
                   center=371,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=41, Center=371+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +409.993+12.670j
SpectralProjNG:   Factorizing at z = +395.099+33.170j
SpectralProjNG:   Factorizing at z = +371.000+41.000j
SpectralProjNG:   Factorizing at z = +346.901+33.170j
SpectralProjNG:   Factorizing at z = +332.007+12.670j
SpectralProjNG:   Factorizing at z = +332.007-12.670j
SpectralProjNG:   Factorizing at z = +346.901-33.170j
SpectralProjNG:   Factorizing at z = +371.000-41.000j
SpectralProjNG:   Factorizing at z = +395.099-33.170j
SpectralProjNG:   Factorizing at z = +409.993-12.670j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [331.93131947 342.80330615 382.23236453 389.52110326 399.28512429
 410.57622274 411.95089915 420.32897117]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [20]:
X.ndof 

16221

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [21]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(95.35112830459103+0j) (99.99999999999983+0j)
(95.14093592373668+0j) (99.99999999999993+0j)
(94.86156035482742+0j) (99.9999999999999+0j)
(94.5433912759974+0j) (100.00000000000009+0j)
(94.62615026412533+0j) (99.99999999999989+0j)
(95.13367889565964+0j) (99.9999999999993+0j)


### This section is to calculate the eigenpairs for H(F)

In [46]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(0.0,0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N
# fesm.ndof
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 19, center = 139; When h=0.03, radius = 20, center = 140; When h=0.05, radius = 41, center = 371
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=29,
                   center=139,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=29, Center=139+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +166.581 +8.961j
SpectralProjNG:   Factorizing at z = +156.046+23.461j
SpectralProjNG:   Factorizing at z = +139.000+29.000j
SpectralProjNG:   Factorizing at z = +121.954+23.461j
SpectralProjNG:   Factorizing at z = +111.419 +8.961j
SpectralProjNG:   Factorizing at z = +111.419 -8.961j
SpectralProjNG:   Factorizing at z = +121.954-23.461j
SpectralProjNG:   Factorizing at z = +139.000-29.000j
SpectralProjNG:   Factorizing at z = +156.046-23.461j
SpectralProjNG:   Factorizing at z = +166.581 -8.961j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [129.36940422 130.46294511 147.52173317 149.39898875 166.67453736
 166.93588372 186.55053967 186.9545824 ]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [47]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(45.31530237341835+0j) (44.87906119627637+0j)
(44.78267957536476+0j) (44.33841352137012+0j)
(58.66912042746918+0j) (58.82806258962528+0j)
(58.659635399371254+0j) (58.887624793401024+0j)
(43.58605126463939+0j) (43.04829021184063+0j)
(43.50340051815427+0j) (42.940370626623164+0j)


## Use Neumann BC

### This section is to calculate the eigenpairs for H(A)

In [26]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(0.0,0.5,0.1).Reverse().Face()
domain.edges.Max(Y).name = 'T'
domain.edges.Max(X).name = 'R'
domain.edges.Min(Y).name = 'B'
domain.edges.Min(X).name = 'L'

geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = 'T|R|B|L')
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

seed =1
npts=10
nspan=10
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 21, center = 115; When h=0.03, radius = 11, center = 153; When h=0.05, radius = 41, center = 371
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=41,
                   center=371,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=41, Center=371+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +409.993+12.670j
SpectralProjNG:   Factorizing at z = +395.099+33.170j
SpectralProjNG:   Factorizing at z = +371.000+41.000j
SpectralProjNG:   Factorizing at z = +346.901+33.170j
SpectralProjNG:   Factorizing at z = +332.007+12.670j
SpectralProjNG:   Factorizing at z = +332.007-12.670j
SpectralProjNG:   Factorizing at z = +346.901-33.170j
SpectralProjNG:   Factorizing at z = +371.000-41.000j
SpectralProjNG:   Factorizing at z = +395.099-33.170j
SpectralProjNG:   Factorizing at z = +409.993-12.670j

=========== Starting FEAST iterations ===========
Trying with 10 vectors:

 ITERATION 1 with 10 vectors
   Real part of computed eigenvalues:
   [333.01378747 342.80674577 381.61140352 389.02444029 399.6380856
 410.47139619 410.67453124 410.97626801 416.79481197 421.88019474]
   Relative 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [27]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(95.35112826709292+0j) (99.99999999999973+0j)
(95.14093588460719+0j) (99.99999999999969+0j)
(94.86156031907188+0j) (100.00000000000004+0j)
(94.54338723256288+0j) (99.99999999999986+0j)
(94.62614015012522+0j) (99.99999999999984+0j)
(95.13366547099783+0j) (99.99999999999976+0j)


### This section is to calculate the eigenpairs for H(F)

In [53]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(0.0,0.5,0.1).Reverse().Face()
domain.edges.Max(Y).name = 'T'
domain.edges.Max(X).name = 'R'
domain.edges.Min(Y).name = 'B'
domain.edges.Min(X).name = 'L'

geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = 'T|R|B|L')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=10
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 21, center = 115; When h=0.03, radius = 22, center = 115; When h=0.05, radius = 25, center = 125
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=25,
                   center=125,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=25, Center=125+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +148.776 +7.725j
SpectralProjNG:   Factorizing at z = +139.695+20.225j
SpectralProjNG:   Factorizing at z = +125.000+25.000j
SpectralProjNG:   Factorizing at z = +110.305+20.225j
SpectralProjNG:   Factorizing at z = +101.224 +7.725j
SpectralProjNG:   Factorizing at z = +101.224 -7.725j
SpectralProjNG:   Factorizing at z = +110.305-20.225j
SpectralProjNG:   Factorizing at z = +125.000-25.000j
SpectralProjNG:   Factorizing at z = +139.695-20.225j
SpectralProjNG:   Factorizing at z = +148.776 -7.725j

=========== Starting FEAST iterations ===========
Trying with 10 vectors:

 ITERATION 1 with 10 vectors
   Real part of computed eigenvalues:
   [100.83618805 105.87205692 129.36826522 130.46055677 147.51972655
 149.39647442 185.70960942 186.63599071 187.33353361 189.98158651]
   Relative

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [54]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(45.31530237341835+0j) (44.879061196276375+0j)
(44.782679575364774+0j) (44.33841352137014+0j)
(58.66912042746917+0j) (58.828062589625276+0j)
(58.65963539937127+0j) (58.88762479340102+0j)
(43.58605126463937+0j) (43.04829021184063+0j)
(43.503400518154265+0j) (42.94037062662317+0j)


# Two holes

### This section is to calculate the eigenpairs for H(A)

In [30]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(-0.8,-0.5,0.1).Reverse().Circle(0.8,-0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
# X.ndof
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 39, center = 133; When h=0.03, radius = 47, center = 161; When h=0.05, radius = 57, center = 362
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=57,
                   center=362,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=57, Center=362+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +416.210+17.614j
SpectralProjNG:   Factorizing at z = +395.504+46.114j
SpectralProjNG:   Factorizing at z = +362.000+57.000j
SpectralProjNG:   Factorizing at z = +328.496+46.114j
SpectralProjNG:   Factorizing at z = +307.790+17.614j
SpectralProjNG:   Factorizing at z = +307.790-17.614j
SpectralProjNG:   Factorizing at z = +328.496-46.114j
SpectralProjNG:   Factorizing at z = +362.000-57.000j
SpectralProjNG:   Factorizing at z = +395.504-46.114j
SpectralProjNG:   Factorizing at z = +416.210-17.614j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [308.98438228 340.32615207 357.24870426 397.94663252 401.72682474
 418.67712069 424.06813466 431.86476922]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [32]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(95.02743736603253+0j) (99.99999999999986+0j)
(95.01912021218173+0j) (100.00000000000001+0j)
(95.65430191956288+0j) (99.99999999999986+0j)
(94.59085049783046+0j) (99.99999999999999+0j)
(94.45056530185012+0j) (100+0j)
(95.6718049625599+0j) (100.00000000000001+0j)


### This section is to calculate the eigenpairs for H(F)

In [58]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Circle(-0.8,-0.5,0.1).Reverse().Circle(0.8,-0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=10
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 39, center = 133; When h=0.03, radius = 40, center = 134; When h=0.05, radius = 47, center = 140
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=40,
                   center=134,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=40, Center=134+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +172.042+12.361j
SpectralProjNG:   Factorizing at z = +157.511+32.361j
SpectralProjNG:   Factorizing at z = +134.000+40.000j
SpectralProjNG:   Factorizing at z = +110.489+32.361j
SpectralProjNG:   Factorizing at z = +95.958+12.361j
SpectralProjNG:   Factorizing at z = +95.958-12.361j
SpectralProjNG:   Factorizing at z = +110.489-32.361j
SpectralProjNG:   Factorizing at z = +134.000-40.000j
SpectralProjNG:   Factorizing at z = +157.511-32.361j
SpectralProjNG:   Factorizing at z = +172.042-12.361j

=========== Starting FEAST iterations ===========
Trying with 10 vectors:

 ITERATION 1 with 10 vectors
   Real part of computed eigenvalues:
   [ 96.5608312  121.8920654  146.71003267 148.11523251 185.01917194
 186.55507783 186.65457379 186.84681231 189.5911337  190.78343923]
   Relative H

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \psi$ and $F\psi$

In [59]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(36.07422132031206+0j) (37.65015728567568+0j)
(35.78544560297917+0j) (37.57410080350771+0j)
(60.68319999617978+0j) (59.07318306588767+0j)
(60.44246613014333+0j) (58.97381783956065+0j)
(44.61567861845536+0j) (44.85939918497072+0j)
(44.447473035275834+0j) (44.71292886968594+0j)


# Three holes

### This section is to calculate the eigenpairs for H(A)

In [36]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).MoveTo(-0.05,0.40).Rectangle(0.2,0.2).Reverse().Circle(-0.8,-0.5,0.1).Reverse().Circle(0.8,-0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 18, center = 153; When h=0.03, radius = 20, center = 186; When h=0.05, radius = 34, center = 384
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius= 34,
                   center= 384,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=34, Center=384+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +416.336+10.507j
SpectralProjNG:   Factorizing at z = +403.985+27.507j
SpectralProjNG:   Factorizing at z = +384.000+34.000j
SpectralProjNG:   Factorizing at z = +364.015+27.507j
SpectralProjNG:   Factorizing at z = +351.664+10.507j
SpectralProjNG:   Factorizing at z = +351.664-10.507j
SpectralProjNG:   Factorizing at z = +364.015-27.507j
SpectralProjNG:   Factorizing at z = +384.000-34.000j
SpectralProjNG:   Factorizing at z = +403.985-27.507j
SpectralProjNG:   Factorizing at z = +416.336-10.507j
Prepare time 0.7842659950256348

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [350.35658342 395.79086196 407.27997488 415.606755   415.79076025
 416.28259498 437.26674508 438.2631018 ]
   Rela

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [37]:
X.ndof 

15268

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [38]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(95.89010220862546+0j) (100.00000000000001+0j)
(94.98985175362371+0j) (99.99999999999996+0j)
(94.8669900122932+0j) (100.00000000000001+0j)
(95.04526131237928+0j) (100.00000000000006+0j)
(95.03582042884022+0j) (99.99999999999994+0j)
(95.01225432271629+0j) (99.99999999999989+0j)


### This section is to calculate the eigenpairs for H(F)

In [60]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).MoveTo(-0.05,0.40).Rectangle(0.2,0.2).Reverse().Circle(-0.8,-0.5,0.1).Reverse().Circle(0.8,-0.5,0.1).Reverse().Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm) 
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 18, center = 153; When h=0.03, radius = 25, center = 159; When h=0.05, radius = 22, center = 168
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=22,
                   center=168,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
print('Prepare time', start2-start1)
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=22, Center=168+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +188.923 +6.798j
SpectralProjNG:   Factorizing at z = +180.931+17.798j
SpectralProjNG:   Factorizing at z = +168.000+22.000j
SpectralProjNG:   Factorizing at z = +155.069+17.798j
SpectralProjNG:   Factorizing at z = +147.077 +6.798j
SpectralProjNG:   Factorizing at z = +147.077 -6.798j
SpectralProjNG:   Factorizing at z = +155.069-17.798j
SpectralProjNG:   Factorizing at z = +168.000-22.000j
SpectralProjNG:   Factorizing at z = +180.931-17.798j
SpectralProjNG:   Factorizing at z = +188.923 -6.798j
Prepare time 0.7764322757720947

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [146.43597234 152.03486195 158.89432527 186.60427105 186.96962661
 189.84210053 190.88587537 191.59308441]
   Rela

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \psi$ and $F\psi$

In [42]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(35.98403991292611+0j) (37.32754837443976+0j)
(35.6726826954233+0j) (37.35999191831952+0j)
(61.326397584076695+0j) (59.79301122080416+0j)
(61.32113730497403+0j) (59.939231077381315+0j)
(44.57092037714351+0j) (44.24635661332306+0j)
(44.93809799854244+0j) (44.30944181002821+0j)
